In [ ]:
# Importing Libraries

import sys
import re
import numpy as np
import tensorflow as tf
import os

from keras.layers.normalization import BatchNormalization 
from keras import activations 
from keras.layers import MaxPool1D, MaxPool2D, Activation, Conv1D, InputLayer, Conv2D, Dense, Dropout, Flatten, LSTM, Bidirectional
from keras.models import Sequential
from tensorflow import keras
import tqdm
import random
from random import choices
from tqdm.notebook import tqdm

In [ ]:
seq_length = 236
motif_length = 20
temp_bp = ' '
prev_bp = ''
sequences_char = list()



# Generating random sequences of length  200
for i in range(2500):
    sequence_char = np.array([])

    
    for j in range(seq_length):
      
        num = random.randint(0, 255) 
        if num <64 and prev_bp != 'A':
            temp_bp = 'A'
            prev_bp = 'A'
        elif num < 128 and num >= 64 and prev_bp != 'T':
            temp_bp = 'T'
            prev_bp = 'T'
        elif num < 192 and num >= 128 and prev_bp != 'C':
            temp_bp = 'C'
            prev_bp = 'C'
        elif num <255 and num >=192 and prev_bp != 'G':
            temp_bp = 'G'
            prev_bp = 'G'
        else:
            num = random.randint(0, 255)
        #print("Current bp:", temp_bp, "Prev: ", prev_bp)
        sequence_char = np.hstack((sequence_char,temp_bp))

        
    sequences_char.append(sequence_char)

    
    
sequences_str=list()
sequences_str2=list()
motif = "AAAAAAAAAAAAAAAAAAAA"
enhancer = "GGGGGGGGGGGGGGGGGGGG"
len(motif)



# Function to convert lists to strings
def listToString(s):  
    # initialize an empty string 
    str1 = " " 
    # return string   
    return (str1.join(s)) 
        

# Combining enhancers along with DNA sequences    
def insert_str(string, str_to_insert, index):
    return string[:index] + str_to_insert + string[index:]

sequences_char2 = sequences_char
for i in range(len(sequences_char)):
    s = sequences_char[i]
    s = listToString(s)
    s = ''.join(s.split())
    num = random.randint(0, 200)
    ss=insert_str(s, enhancer,num)
    s=insert_str(s, motif,num)
    sequences_str.append(s)
    sequences_str2.append(ss)

text_file = open("SEQ.FASTA", "w")

for i in tqdm(range(2500)):
  temp_str = sequences_str[i]
  text_file.write('>'+'\n'+ temp_str + '\n')


for i in range(2500):
  temp_str = sequences_str2[i]
  text_file.write('>'+ '\n'+temp_str+ '\n')

  

text_file.close()


In [ ]:

# Providing the data to be converted into one hot encoded vectors
fileName = 'SEQ'
num_samples = 5000
seq_length = 256
BASE_FILE_DIR = "/content/"
Obj = open(BASE_FILE_DIR+"/"+fileName+".FASTA")

def lessThanThreshold():
  return np.random.uniform(low=0.01, high=0.39, size = 1)[0]
   

def greaterThanThreshold():
  return np.random.uniform(low=0.6, high=1.0, size = 1)[0]

def baseToOnehot(base):
    
    if base == 'A':
        return np.array([greaterThanThreshold(),lessThanThreshold(),lessThanThreshold(),lessThanThreshold()])
    elif base == 'G':
        return np.array([lessThanThreshold(),greaterThanThreshold(),lessThanThreshold(),lessThanThreshold()])
    elif base == 'C':
        return np.array([lessThanThreshold(),lessThanThreshold(),greaterThanThreshold(),lessThanThreshold()])
    elif base == 'T':
        return np.array([lessThanThreshold(),lessThanThreshold(),lessThanThreshold(),greaterThanThreshold()])


In [ ]:
# Creating one hot encoded numpy arrays of length seq_length
all_samples = np.zeros((num_samples, seq_length, 4))
current_sample = np.zeros((seq_length,4))

sample_count = 0

for line in Obj:
    line = line.strip('\r\n')
    if re.search('>',line):
      sample_count += 1
    else:
        line = line.upper()
        first_base = True
        base_count = 0
        for base in line:
            if first_base == True:
                first_base = False
                current_sample = np.zeros((seq_length,4))
                current_sample[base_count] = baseToOnehot(base)
                base_count += 1
            else:
                current_sample[base_count] = baseToOnehot(base)
                base_count += 1
                all_samples[sample_count-1] = current_sample
np.save(BASE_FILE_DIR+"/"+fileName+".npy", all_samples)
print(fileName + " Done.")

SEQ Done.


In [ ]:
# Playing with the shape of the data 
print("Shape of the data before expanding dimensions", all_samples.shape)
#all_samples = np.expand_dims(all_samples, axis=0)
#print("Shape of the data after expanding dimensions", all_samples.shape)
#all_samples = np.reshape(all_samples, (20,4,1,seq_length))
#print("Shape of the data after Reshaping", all_samples.shape)
#np.set_printoptions(threshold=np.inf)
all_samples = np.reshape(all_samples, (num_samples,seq_length,4))
print(all_samples.shape)

# final_samples = np.reshape (all_samples, (num_samples,seq_length,4,1))
final_samples = all_samples

print(final_samples.shape)

Shape of the data before expanding dimensions (5000, 256, 4)
(5000, 256, 4)
(5000, 256, 4)


In [ ]:
final_samples = final_samples.astype('float32')

# Discriminator RNN

In [ ]:
inp = keras.Input(shape=(256, 4))
x = LSTM(128, return_sequences=True)(inp)
x = Bidirectional(LSTM(128))(x)
x = Dense(128)(x)
x = keras.layers.LeakyReLU(alpha=0.2)(x)
x = Dense(64)(x)
x = keras.layers.LeakyReLU(alpha=0.2)(x)
out = Dense(1, activation='sigmoid')(x)

discriminator = keras.Model(inp, out)

In [ ]:
discriminator.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 4)]          0         
_________________________________________________________________
lstm (LSTM)                  (None, 256, 128)          68096     
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               263168    
_________________________________________________________________
dense (Dense)                (None, 128)               32896     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 64)               

In [ ]:
# keep tuning learning rate 0.0001, 0.0005, 0.001
discriminator.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0005), loss='binary_crossentropy')
discriminator.trainable=False

# Generator RNN

In [ ]:
g_inp = keras.Input(shape=(100,))

x_hat =  keras.layers.Dense(256)(g_inp)
x_hat = keras.layers.LeakyReLU(0.2)(x_hat)
x_hat = keras.layers.BatchNormalization(momentum=0.8)(x_hat)
x_hat =  keras.layers.Dense(512)(x_hat)
x_hat = keras.layers.LeakyReLU(0.2)(x_hat)
x_hat = keras.layers.BatchNormalization(momentum=0.8)(x_hat)
x_hat =  keras.layers.Dense(1024)(x_hat)
x_hat = keras.layers.LeakyReLU(0.2)(x_hat)
x_hat = keras.layers.BatchNormalization(momentum=0.8)(x_hat)
x_hat =  keras.layers.Dense(256*4, activation='sigmoid')(x_hat)
g_out = keras.layers.Reshape(target_shape=(256, 4))(x_hat)

generator = keras.Model(g_inp, g_out)


In [ ]:
generator.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
dense_4 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)              

In [ ]:
# Making the complete Gan Model 
# keep tuning learning rate 0.0001, 0.0005, 0.001
gan = keras.models.Sequential([generator, discriminator])
gan.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy')

In [ ]:
final_samples
dna_seqs = tf.data.Dataset.from_tensor_slices(final_samples).batch(32, drop_remainder=True)


def train_GAN(gan, seqs, batch_size, num_features, epochs):
  generator, discriminator = gan.layers
  for epoch in tqdm(range(epochs)):
    print("Epoch {}/{}".format( epoch, epochs))

    # Taking batch of sequences
    for X_batch in seqs:
      
      # Generating noise for a batch
      noise = tf.random.normal(shape=(batch_size, num_features))
      
      generated_seqs = generator(noise)
      
      # Combining the real and generated seqs from the generator
      real_and_fake_seqs = tf.concat((generated_seqs, X_batch), axis =0)

      # passing the data to the discriminator
      discriminator.trainable = True
      y = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
      
      discriminator.train_on_batch(real_and_fake_seqs, y)

      # Training the GAN

      y_dash = tf.constant([[1.0]] * batch_size)
      discriminator.trainable = False
      gan.train_on_batch(noise, y_dash)


    print(discriminator.evaluate(real_and_fake_seqs, y))

In [ ]:
train_GAN(gan, dna_seqs, batch_size = 32, num_features = 100, epochs= 50)

Epoch 0/50
2/2 [==============================] - 0s 9ms/step - loss: 0.2596
0.2595638930797577
Epoch 1/50
2/2 [==============================] - 0s 11ms/step - loss: 0.5741
0.5741366744041443
Epoch 2/50
2/2 [==============================] - 0s 12ms/step - loss: 0.6634
0.6633647680282593
Epoch 3/50
2/2 [==============================] - 0s 9ms/step - loss: 0.3839
0.38389265537261963
Epoch 4/50
2/2 [==============================] - 0s 10ms/step - loss: 0.2420
0.24198448657989502
Epoch 5/50
2/2 [==============================] - 0s 9ms/step - loss: 0.6961
0.6960759162902832
Epoch 6/50
2/2 [==============================] - 0s 10ms/step - loss: 0.2562
0.25618433952331543
Epoch 7/50
2/2 [==============================] - 0s 9ms/step - loss: 0.6588
0.6588290929794312
Epoch 8/50
2/2 [==============================] - 0s 9ms/step - loss: 0.6881
0.6880876421928406
Epoch 9/50
2/2 [==============================] - 0s 9ms/step - loss: 0.6691
0.6691018342971802
Epoch 10/50
2/2 [================

In [ ]:
z = np.random.normal( size=[1,100])
arr = generator.predict(z)
arr

array([[[0.00439637, 0.52587533, 0.9994405 , 0.17459519],
        [0.04636385, 0.5659289 , 0.90807164, 0.0177791 ],
        [0.47881326, 0.72188175, 0.24348207, 0.19725266],
        ...,
        [0.12487207, 0.534596  , 0.4005842 , 0.5995917 ],
        [0.47062984, 0.44543186, 0.48673612, 0.12545533],
        [0.5189485 , 0.13272376, 0.07274412, 0.77011925]]], dtype=float32)

In [ ]:
arr2 = np.argmax(arr, axis=2)


In [ ]:
arr2

array([[2, 2, 1, 0, 2, 2, 2, 1, 3, 1, 0, 1, 2, 0, 2, 1, 3, 1, 3, 1, 3, 1,
        0, 0, 1, 3, 2, 2, 1, 1, 3, 1, 2, 1, 2, 2, 2, 3, 2, 3, 1, 0, 0, 2,
        0, 2, 2, 2, 0, 1, 0, 0, 1, 2, 0, 1, 1, 3, 2, 3, 1, 2, 3, 1, 1, 2,
        1, 0, 2, 1, 2, 2, 0, 0, 3, 1, 1, 0, 3, 0, 0, 3, 1, 1, 2, 0, 0, 3,
        0, 1, 0, 3, 0, 3, 0, 1, 0, 0, 0, 0, 3, 1, 3, 3, 2, 1, 1, 0, 2, 1,
        2, 0, 3, 3, 0, 2, 3, 2, 0, 2, 1, 3, 2, 0, 2, 2, 0, 1, 0, 1, 3, 2,
        0, 3, 3, 1, 3, 1, 0, 2, 1, 3, 1, 1, 1, 1, 1, 0, 0, 1, 1, 2, 1, 0,
        1, 3, 0, 2, 0, 0, 3, 1, 3, 3, 1, 2, 0, 0, 2, 1, 0, 3, 2, 0, 1, 3,
        3, 3, 2, 3, 1, 1, 3, 2, 0, 1, 2, 0, 0, 3, 0, 2, 3, 3, 0, 3, 0, 0,
        2, 2, 3, 1, 1, 2, 2, 1, 0, 1, 2, 2, 0, 2, 1, 0, 1, 2, 1, 1, 0, 1,
        1, 0, 3, 3, 1, 3, 3, 1, 3, 1, 1, 3, 1, 1, 1, 2, 1, 3, 0, 0, 0, 0,
        0, 0, 0, 1, 3, 2, 0, 2, 0, 0, 2, 3, 2, 3]])

In [ ]:
idcies = np.reshape(arr2, (256))
list(idcies)
bxr = np.zeros_like(arr)
for i, g in enumerate(idcies):
  bxr[0][i][g] = 1

In [ ]:
generator.save('genRNN.h5')

In [ ]:
bxr

array([[[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        ...,
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]], dtype=float32)

In [ ]:
bxr.shape

(1, 256, 4)

In [ ]:
arr = np.reshape(bxr, (256,4))

In [ ]:
arr

array([[0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)

In [ ]:
for i in range (256):
  temp = arr[i][:]
  n=np.argmax(temp)
  if n==0:
    print('A', end='')
  elif n== 1:
    print('G', end='')
  elif n== 2:
    print('C', end='')
  elif n== 3:
    print('T', end='')

CCGACCCGTGAGCACGTGTGTGAAGTCCGGTGCGCCCTCTGAACACCCAGAAGCAGGTCTGCTGGCGACGCCAATGGATAATGGCAATAGATATAGAAAATGTTCGGACGCATTACTCACGTCACCAGAGTCATTGTGACGTGGGGGAAGGCGAGTACAATGTTGCAACGATCAGTTTCTGGTCAGCAATACTTATAACCTGGCCGAGCCACGAGCGGAGGATTGTTGTGGTGGGCGTAAAAAAAGTCACAACTCT

In [ ]:
AGCT